In [1]:
!pip install ipywidgets


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install transformers datasets torch pillow


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoModelForCausalLM, AutoProcessor

In [4]:
import torch

torch.set_num_threads(4) # intraop parallelism on CPU
torch.set_num_interop_threads(4) # interop parallelism on CPU
print("default num threads for intraop parallelism: ", torch.get_num_threads())
print("default num threads for interop parallelism: ", torch.get_num_interop_threads())

default num threads for intraop parallelism:  4
default num threads for interop parallelism:  4


In [5]:
blip_model = BlipForConditionalGeneration.from_pretrained("../models/blip-large/")

In [6]:
blip_model

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-23): 24 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (projection): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (layer_norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((1024,),

In [7]:
blip_model.num_parameters()

469732924

In [8]:
git_model = AutoModelForCausalLM.from_pretrained("../models/git-large/")

In [9]:
git_model.num_parameters()

394196026

In [10]:
git_model

GitForCausalLM(
  (git): GitModel(
    (embeddings): GitEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (image_encoder): GitVisionModel(
      (vision_model): GitVisionTransformer(
        (embeddings): GitVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
          (position_embedding): Embedding(257, 1024)
        )
        (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (encoder): GitVisionEncoder(
          (layers): ModuleList(
            (0-23): 24 x GitVisionEncoderLayer(
              (self_attn): GitVisionAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
      

In [11]:
blip_processor = BlipProcessor.from_pretrained("../models/blip-large/")
git_processor = AutoProcessor.from_pretrained("../models/git-large/")

In [12]:
from PIL import Image

image_path = '/Users/naormatania/code/image-captioning/datasets/website-screenshots/test/file_71.jpg'
img = Image.open(image_path)

In [19]:
%%time
blip_inputs = blip_processor(images=img, return_tensors="pt")
blip_out = blip_model.generate(**blip_inputs)

CPU times: user 8.94 s, sys: 326 ms, total: 9.26 s
Wall time: 8.91 s


In [14]:
%%time
blip_inputs = blip_processor(images=[img]*4, return_tensors="pt")
blip_out = blip_model.generate(**blip_inputs)

/Users/naormatania/code/image-captioning/notebooks/my-env/lib/python3.11/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


CPU times: user 35.3 s, sys: 2.45 s, total: 37.8 s
Wall time: 34.4 s


In [15]:
%%time
git_inputs = git_processor(images=img, return_tensors="pt")
git_out = git_model.generate(**git_inputs)

CPU times: user 13.5 s, sys: 3.29 s, total: 16.8 s
Wall time: 19.1 s


In [16]:
%%time
git_inputs = git_processor(images=[img]*4, return_tensors="pt")
git_out = git_model.generate(**git_inputs)

CPU times: user 48.9 s, sys: 3.12 s, total: 52 s
Wall time: 46.7 s


In [21]:
compiled_blip = torch.compile(blip_model)
compiled_git = torch.compile(git_model)

In [22]:
%%time
blip_out = compiled_blip.generate(**blip_inputs)

CPU times: user 34.6 s, sys: 1.62 s, total: 36.3 s
Wall time: 32 s


In [23]:
%%time
git_out = compiled_git.generate(**git_inputs)

CPU times: user 48.1 s, sys: 2.65 s, total: 50.8 s
Wall time: 44.4 s


In [24]:
import threading
threading.activeCount()

/var/folders/qt/bczh05655z57blf3r39zzh780000gn/T/ipykernel_90279/1560711055.py:2: DeprecationWarning: activeCount() is deprecated, use active_count() instead
  threading.activeCount()


9

In [26]:
import os
os.cpu_count() 

8

In [27]:
import multiprocessing
multiprocessing.cpu_count()

8

In [20]:
dir(git_processor)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_class',
 '_create_repo',
 '_get_arguments_from_pretrained',
 '_get_files_timestamps',
 '_upload_modified_files',
 'attributes',
 'batch_decode',
 'current_processor',
 'decode',
 'feature_extractor_class',
 'from_args_and_dict',
 'from_pretrained',
 'get_processor_dict',
 'image_processor',
 'image_processor_class',
 'model_input_names',
 'push_to_hub',
 'register_for_auto_class',
 'save_pretrained',
 'to_dict',
 'to_json_file',
 'to_json_string',
 'tokenizer',
 'tokenizer_class']

In [22]:
git_processor.tokenizer.tokenize('a screenshot of a website')

['a', 'screens', '##hot', 'of', 'a', 'website']

In [23]:
blip_processor.tokenizer.tokenize('a screenshot of a website')

['a', 'screens', '##hot', 'of', 'a', 'website']

In [30]:
from transformers import AutoModelConditionalGeneration

bm = AutoModelConditionalGeneration.from_pretrained("../models/blip-large/")

ImportError: cannot import name 'AutoModelConditionalGeneration' from 'transformers' (/Users/naormatania/code/image-captioning/notebooks/my-env/lib/python3.11/site-packages/transformers/__init__.py)

In [35]:
from datasets import load_dataset

ds = load_dataset("ydshieh/coco_dataset_script", "2017", data_dir="/Users/naormatania/code/image-captioning/datasets")
ds

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 591753
    })
    validation: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 25014
    })
    test: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 40670
    })
})

In [38]:
import os

print(os.path.basename(ds['validation'][0]['image_path']))
print(ds['validation'][0]['caption'])

000000179765.jpg
A black Honda motorcycle parked in front of a garage.


In [40]:
os.cwd

AttributeError: module 'os' has no attribute 'cwd'

In [2]:
from transformers import BlipProcessor, AutoProcessor
processor = BlipProcessor.from_pretrained("../models/blip-large/")

In [3]:
processor.tokenizer

BertTokenizerFast(name_or_path='../models/blip-large/', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}